<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Modification of the fuselage cabin length of an aircraft.

This notebook aims to compare a reference architecture represented by the .xml file of an aircraft (Beechcraft 76, Cirrus 22, TBM-900) with a modified configuration. The modified architecture is produced by the FAST-OAD-GA module called modify_config, which creates a new .xml file based on the reference file. The .xml file of the modified architecture then goes through all of the FAST-OAD-GA modules. Once it is done, the consequences of the geometrical changes on the aerodynamics, the performance and the weight of the aircraft can be compared with the reference aircraft in the post-processing notebook.

The reference aircraft is preferably the output of a MultiDisciplinary Analysis. Please note that if the relative tolerance of the MDA is quite rough (rtol >= 1e3) then it is probable that the aircraft will undergo visible geometrical modifications when passed through the Geometry module. These modifications have nothing to do with the present use case and to avoid them it is preferable to perform a MDA with a higher accuracy (rtol < 1e4). In that way the only visible modifications will be the one intended by the use case at hand and the rest of the aircraft geometry will remain untouched. The use of the dedicated notebook "MDA (adapted for analysis mode)" will prevent the user from encountering these issues.

The aircraft modification computed and studied in this notebook is the increase of the fuselage cabin length. The user has to specify this length, or can choose to add a row of seats. The user can also play on other parameters :
- The relative position of the added section with respect to the wings (in front of or behind the wings)
- Add passengers to the design mission
- Add luggage mass to the design mission

Like in the tutorials, the structure of the notebook is based on data transfers between different folders :
- The data folder stocks the MDA outputs of the reference aircrafts. These files serve as reference files for the comparison.
- The workdir folder stocks the .xml files of the modified architectures passing through the modules of FAST-OAD-GA.
- The output folder stocks the final .xml files, which are ready for post-processing.

For a proper notebook execution, the use of the analysis_mode branch of FAST-GA-OAD is strongly recommended.

If the user wants to perform a new analysis after the notebook cells have already been run, for any change in the widget values, in the configuration files or in the aircraft modification parameters it is advised to restart the kernel to avoid unwelcomed inconsistencies.

## 1. Initialization of the work environment

### Choice of the aircraft

The available aircrafts are :

- Beechcraft Duchess 76
- Cirrus SR22
- TBM 930

The Beechcraft 76 is the default option.

In [1]:
from ipywidgets import widgets, Layout
from fastoad.io import VariableIO
from IPython.display import display, clear_output

liste = widgets.Dropdown(
    options=[('Beechcraft Duchess 76', 'output_mda_beech.xml'), ('Cirrus SR22', 'output_mda_cirrus_sr22.xml'), ('TBM-930', 'output_mda_tbm930.xml')],
    value='output_mda_beech.xml',
    description='Reference Aircraft:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the reference aircraft
ref_aircraft = 'output_mda_beech.xml'
print('The reference aircraft .xml file is '+ ref_aircraft)

def eventhandler(change):
    if (change.new):
        global ref_aircraft
        ref_aircraft = liste.value
        print('The reference aircraft .xml file is '+ ref_aircraft)

liste.observe(eventhandler, names='value')

display(liste)

Failed to import module fastga.models.weight.cg.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.performances.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.handling_qualities.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.weight.cg.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.aerodynamics.unitary_tests.test_functions.py
Failed to import module fastga.models.propulsion.fuel_propulsion.turboprop_simple.unitary_tests.test_openmdao_engine.py
Failed to import module fastga.models.geometry.unitary_tests.test_beechcraft_76.py
Failed to import module fastga.models.geometry.unitary_tests.test_cirrus_sr22.py
Failed to import module fastga.models.loops.unitary_tests.test_wing_area.py
Failed to import module fastga.models

The reference aircraft .xml file is output_mda_beech.xml


Dropdown(description='Reference Aircraft:', layout=Layout(width='30%'), options=(('Beechcraft Duchess 76', 'ou…

### Choice of the engine model

The available models are :

- Basic IC Engine
- Basic Turboprop

The basic IC engine is the default option.

In [2]:
liste_engine = widgets.Dropdown(
    options=[('Basic IC Engine', "fastga.wrapper.propulsion.basicIC_engine"), ('Basic Turboprop', "fastga.wrapper.propulsion.basic_turboprop")],
    value="fastga.wrapper.propulsion.basicIC_engine",
    description='Propulsion Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='30%'),
)

# Initialization of the engine model
engine_id = "fastga.wrapper.propulsion.basicIC_engine"
print('The engine model id chosen is '+ engine_id)

def eventhandler(change):
    if (change.new):
        global engine_id
        engine_id = liste_engine.value
        print('The engine model id chosen is '+ engine_id)

liste_engine.observe(eventhandler, names='value')

display(liste_engine)

The engine model id chosen is fastga.wrapper.propulsion.basicIC_engine


Dropdown(description='Propulsion Model:', layout=Layout(width='30%'), options=(('Basic IC Engine', 'fastga.wra…

### Preparation of files

The reference aircraft file "AIRCRAFT_REF_FILE" is a copy from the corresponding mda output file. The aircraft id depends on the choice made with the widget.

This file is then directly duplicated into "AIRCRAFT_MOD_FILE", which is the file that will receive the fuselage modifications. So after the execution of this cell, both files are identical.

Please note that the engine_id widget allows for the user to skip the part where he has to manually write the engine's id in the parameters of the classes executed by generate_block_analysis. But for the weight/performance loop that uses the weight_loop.yml configuration file the user has to manually check that the right engine's id will be used for the weight and performance modules calculation. If the yml file is modified the changes will only take effect after the kernel is restarted.

In [3]:
import os.path as pth
import os
import openmdao.api as om
from fastoad import api as api_cs25
from fastga.command import api as api_cs23
import logging
from fastoad.gui import VariableViewer
import shutil

# Define relative path.
current_path = os.getcwd()
DATA_FOLDER_PATH = pth.join(current_path, 'data')
WORK_FOLDER_PATH = pth.join(current_path, 'workdir')
OUTPUT_FOLDER_PATH = pth.join(current_path, 'output')

# Clear work folder.
shutil.rmtree(WORK_FOLDER_PATH, ignore_errors=True)
os.mkdir(WORK_FOLDER_PATH)

# For using more screen width.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Final file names. These .xml files will be overwritten each time they pass through a module.
AIRCRAFT_REF_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_reference.xml')
AIRCRAFT_MOD_FILE = pth.join(WORK_FOLDER_PATH, 'geometry_mod.xml')

# Copy the reference aircraft .xml file.
shutil.copy(pth.join(DATA_FOLDER_PATH, ref_aircraft), AIRCRAFT_REF_FILE)
shutil.copy(AIRCRAFT_REF_FILE, AIRCRAFT_MOD_FILE)

# Definition of the files for the weight MDA.
CONFIG_FILE = pth.join(DATA_FOLDER_PATH, 'weight_loop.yml')
WORK_CONFIG_FILE = pth.join(WORK_FOLDER_PATH, 'weight_loop.yml')
shutil.copy(CONFIG_FILE, WORK_CONFIG_FILE)
WEIGHT_LOOP_FILE = pth.join(WORK_FOLDER_PATH, 'problem_inputs_weight_loop.xml')

## 2. Definition of the modifications from the reference architecture

The different parameters the user has to choose in order to obtain the desired modified configuration are:

- **added_length [m]** : If 0, the length of a row of seats will be taken (this length is defined in the xml file).
- **added_section_x_ratio_front** : Float between 0 and 1 defining the percentage of the added section that will be placed ahead of the wing MAC 25%.
- **added_section_x_ratio_rear** : Float between 0 and 1 defining the percentage of the added section that will be placed at the back of the wing MAC 25%.
- **added_pax** : Integer stating the number of passengers added to the design mission. Take care not to exceed npax_max.
- **added_luggage [kg]** : Float stating the mass of luggage added to the design mission.

If a row of seats is added (added_length = 0), then the code will increase the maximum number of passengers. If a specified length is used (for example added_length = 0.5 m), then the code won't increase the maximum number of passengers. In this case the added section will be an empty space in the cabin.

The sum of added_section_x_ratio_front and of added_section_x_ratio_rear must be equal to 1.

There is no limit in the code for the increase in cabin length, but some modules could have problems running with an extreme value.
A cabin length reduction can also be theorically computed by specifying a negative length, but the consequences on the code have not been studied. 

In [4]:
# Cabin length increase parameters
added_length = 0
added_section_x_ratio_front = 0
added_section_x_ratio_rear = 1
added_pax = 2
added_luggage = 0

fuselage_mod_parameters = [added_length, added_section_x_ratio_front, added_section_x_ratio_rear, added_pax, added_luggage]

Once these parameters are defined, they can be used as options in the ComputeConfigMod class of the modify_config module.

This module computes the new geometrical quantities that redefine the wing. These modified values are stocked as data_mod quantities in the .xml files. The use of the variable viewer allows to quickly understand what has changed with respect to the reference architecture.

In [5]:
from fastga.models.modify_config import ComputeConfigMod


compute_modify = api_cs23.generate_block_analysis(
        ComputeConfigMod(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_modify({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

The UpdateXML class takes the values from data_mod and copies them into the associated data quantities.

This process highlights one limitation of generate_block_analysis: we cannot modify a quantity that is used as an input of a generate_block_analysis. That is why the architecture modification is computed in two steps: the first one which computes the new values of the geometry, and the second one which updates the old values with the new ones.

In [6]:
from fastga.models.modify_config.update_XML import UpdateXML


compute_update = api_cs23.generate_block_analysis(
        UpdateXML(fuselage_mod=fuselage_mod_parameters),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )

output_mod = compute_update({})

# Open viewer
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

## 3. Computation of the FAST-OAD-GA modules

The FAST-OAD-GA modules are executed in order to compute the modifications resulting from the wing geometry changes of the previous section.

The weight loop is a mda that loops on the weight / update mtow / performance modules in order to get converged MTOW and fuel mission. Its rtol parameter is quite low but it ensures to get consistent values for the operation MTOW = fuel + payload + owe.

The load analysis module is also executed, for further post-processing reasons.

There is no need in this type of analysis to loop at the end of the process because the aim is to observe the characteristics of an aircraft with a given geometry (cabin length increase) and not to optimize it by resizing its tails and wings.

It is advised to change the class parameters with caution as some modules require some of the quantities computed with these parameters on (for example the wing structural analysis requires the computation of the slipstream in the aerodynamic module, and the load factor class requires the mach interpolation computation).

The expected computational time is around 3 minutes. The aerodynamic module is the most demanding part of the analysis.

In [7]:
from fastga.models.geometry import GeometryFixedTailDistance as Geometry
from fastga.models.aerodynamics.aerodynamics import Aerodynamics
from fastga.models.aerodynamics.load_factor import LoadFactor
from fastga.models.load_analysis.loads import Loads
from fastga.models.handling_qualities.handling_qualities import ComputeHandlingQualities


# GEOMETRY
compute_geometry_mod = api_cs23.generate_block_analysis(
        Geometry(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_geometry_mod({})

# AERODYNAMICS
compute_aero_mod = api_cs23.generate_block_analysis(
        Aerodynamics(
            propulsion_id=engine_id,
            use_openvsp=True,
            compute_mach_interpolation=True,
            compute_slipstream_cruise=True,
#             wing_airfoil_file="naca43013_3.af"
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_aero_mod({})

# LOAD FACTOR (ComputeVN and speed identification)
compute_load_factor = api_cs23.generate_block_analysis(
        LoadFactor(propulsion_id=engine_id),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output = compute_load_factor({})

# WEIGHT / MTOW / PERFORMANCES loop

shutil.copy(AIRCRAFT_MOD_FILE, WEIGHT_LOOP_FILE)

eval_problem = api_cs25.evaluate_problem(WORK_CONFIG_FILE, overwrite=True)

OUTPUT_FILE = pth.join(WORK_FOLDER_PATH,'problem_outputs_weight_loop.xml')
shutil.copy(OUTPUT_FILE, AIRCRAFT_MOD_FILE)

# LOAD ANALYSIS
compute_loads_mod = api_cs23.generate_block_analysis(
        Loads(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_loads_mod({})

# HANDLING QUALITIES
compute_hq_mod = api_cs23.generate_block_analysis(
        ComputeHandlingQualities(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output_mod = compute_hq_mod({})

Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261
Area ratio value outside of the range in Roskam's book k_vh, value clipped2.6162514994300965
min value of Roskam's book is 8.164931618692696e-05
max value of Roskam's book is 1.9750765462339261


NL: NLBGS 1 ; 180005859 1
NL: NLBGS 2 ; 10560.5116 5.86675992e-05
NL: NLBGS 3 ; 2435.52572 1.35302581e-05
NL: NLBGS 4 ; 600.838907 3.33788528e-06
NL: NLBGS Converged


In [8]:
api_cs25.variable_viewer(AIRCRAFT_MOD_FILE)

A small post-processing tool to visualize the changes.

In [ ]:
from fastga.utils.postprocessing.analysis_and_plots import aircraft_geometry_plot

fig = aircraft_geometry_plot(AIRCRAFT_REF_FILE, name='reference aircraft', plot_nacelle = False)
fig = aircraft_geometry_plot(AIRCRAFT_MOD_FILE, name='modified aircraft', fig=fig, plot_nacelle = False)
fig.show()

### Running complementary methods

These classes are not part of the sizing loop of the aircraft. Their computation is required for postprocessing reasons.

The expected computational time is around 2 minutes. The payload range is the most demanding part of the analysis. Its progress can be tracked with the display of the number of iterations for each point that uses fsolve (point B, point D and point E).

In [ ]:
from fastga.models.aerodynamics.components import ComputePolar
from fastga.models.performances.payload_range import ComputePayloadRange

# POLAR
compute_polar = api_cs23.generate_block_analysis(
        ComputePolar(),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
    )
output = compute_polar({})

# Payload Range
compute_payload_range = api_cs23.generate_block_analysis(
        ComputePayloadRange(
            propulsion_id=engine_id,
        ),
        [],
        str(AIRCRAFT_MOD_FILE),
        True,
)
output = compute_payload_range({})

## Save files for post processing

The name of the output file has to be manually written. This file is ready to be used in the post-processing notebook. It is useless to save the reference aircraft for post-processing, since the file from the data folder can be directly used.

In [ ]:
OUTPUT_MOD_FILE = pth.join(OUTPUT_FOLDER_PATH, 'output_mod_fuselage_tbm.xml')
shutil.copy(AIRCRAFT_MOD_FILE, OUTPUT_MOD_FILE)

In [ ]:
from fastoad.io import VariableIO

# Static margins
variables_ref = VariableIO(AIRCRAFT_REF_FILE).read()
static_margin_fixed_ref = round(variables_ref["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_ref = round(variables_ref["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("TBM 930\t\t\t\t\t\t", "static margin stick fixed:", static_margin_fixed_ref, "\tstatic margin stick free:", static_margin_free_ref)

variables_mod = VariableIO(AIRCRAFT_MOD_FILE).read()
static_margin_fixed_mod = round(variables_mod["data:handling_qualities:stick_fixed_static_margin"].value[0],4)
static_margin_free_mod = round(variables_mod["data:handling_qualities:stick_free_static_margin"].value[0],4)
print("TBM 930 with fuselage stretched by 1 meter\t", "static margin stick fixed:", static_margin_fixed_mod, "\tstatic margin stick free:", static_margin_free_mod)